In [1]:
#!/usr/bin/env python
"""non_supervise: Advanced Pipeline of TF-IDF."""

__author__      = "Louis-Fiacre Franchet d'Espèrey"
__copyright__   = "Copyright 2022"
__commentary__ = "Adaptated by Perrine Maurel with the explicit agreement of {__author__} and the help of Violette Saïag."

In [2]:
import os
import pandas as pd
import utilities

In [25]:
dir_fr = "../../textes_fr"
dir_en = "../../textes_en"
titres_en = [name_file for name_file in os.listdir(dir_en) if name_file.endswith('.txt')]
titres_fr = [name_file for name_file in os.listdir(dir_fr) if name_file.endswith('.txt')]

In [4]:
list_textes_en = [] #on crée une liste vide pour accueillir nos résultats
for titre in titres_en :
    with open(os.path.join(dir_en,titre), 'r', encoding='utf-8') as file: #on ouvre et on lit le fichier
        list_textes_en.append(file.read())

In [26]:
list_textes_fr = [] #on crée une liste vide pour accueillir nos résultats
for titre in titres_fr :
    with open(os.path.join(dir_fr,titre), 'r', encoding='utf-8') as file: #on ouvre et on lit le fichier
        list_textes_fr.append(file.read())

## On crée le corpus

In [5]:
metadonnees_en = utilities.path_extension("en_trad/bd_en",True,lecture=True)
metadonnees_fr = utilities.path_extension("bd_fr",True,lecture=True)

In [6]:
metadonnees_en

,id,Titre,Auteurice(s),Traducteurice(s),Date de création,Image & Licence,Remerciements,Commentaires,Autres
0,/scp-2994,SCP-2994 - Interactive Whiteboards,Sound Chaser,Dr Dharma,2014-09-26 00:00:00,Non-assigné,Non-assigné,Non-assigné,Non-assigné
1,/scp-6187,SCP-6187 - From Space With Love,Dr Leonerd,Mars la Grise,2022-02-14 00:00:00,Globular star cluster Messier 4 de ESO Imaging...,Non-assigné,Non-assigné,Non-assigné
2,/scp-746,SCP-746 - Pseudo-Avian,"Original par Dexanote, réécrit par Baronjoe",Karnot,2018-07-07 00:00:00,* Contenu original,Non-assigné,Non-assigné,Non-assigné
3,/scp-3681,SCP-3681 - 5 - 0,macro_au_micro,macro_au_micro,2019-03-11 00:00:00,Image du ballon,Non-assigné,Non-assigné,Non-assigné
4,/scp-403,SCP-403 - Escalating Lighter,LitigousReport,Douger-arched,2009-05-25 00:00:00,Informations manquantes,Non-assigné,Non-assigné,Non-assigné
...,...,...,...,...,...,...,...,...,...
3518,/scp-727,SCP-727 - Hephaestus's Forge,Shebleha,D-2108,2011-06-22 00:00:00,Temple,Non-assigné,Non-assigné,Non-assigné
3519,/never-metafictional-character-i-liked,Never Metafictional Character I Didn't Like,The Great Hippo,Felter Finalis,2017-11-25 00:00:00,lien,Non-assigné,Non-assigné,Non-assigné
3520,/detained,Detained,Randomini,TheEulays,2013-12-19 00:00:00,Non-assigné,Non-assigné,Non-assigné,Non-assigné
3521,/threat-of-joy,Threat of Joy,TyGently,Trank,2019-06-24 00:00:00,Non-assigné,Non-assigné,Non-assigné,Non-assigné


## On divise le corpus en sous-corpus à partir des traductions et des textes originaux des traducteurs

In [7]:
import time

In [8]:
def get_traducteur(id):
    res = metadonnees_en.loc[metadonnees_en['id']==id]['Traducteurice(s)']
    index = list(res.index)
    return(res[index[0]])

In [9]:
def get_all_works(author):
    works = []
    for _,row in metadonnees_fr.iterrows():
        if utilities.is_in(author,row.loc['Auteurice(s)']):
            works = utilities.insert_in_list_by_date((row.loc['id'],row.loc['Date de création']),works)
    return(works)


In [10]:
def make_corpus_trad(id):

    traducteur = get_traducteur(id)
    traducteur_works = get_all_works(traducteur)
    date=metadonnees_en.loc[metadonnees_en['id']==id]['Date de création']
    index = list(date.index)
    date = date[index[0]][:10]
    if date[:2] != "20":
        return []
    date = time.strptime(date, '%Y-%m-%d')
    valid_works = [work[0] for work in traducteur_works if work[1]>date]
    if valid_works != []:
        with open('../../textes_en'+id+'.txt',"r",encoding="utf-8") as file:
            text_trad = file.read()
        valid_work_text = [text_trad]
        for text_id in valid_works:
            try:
                open('../../textes_fr'+text_id+'.txt',"r",encoding="utf-8")
            except:
                a=0
            else:
                with open('../../textes_fr'+text_id+'.txt',"r",encoding="utf-8") as file:
                    valid_work_text += [file.read()]
        if len(valid_work_text)>1:
            return(valid_work_text)
        else:
            return []
    else:
        return []

In [11]:
len(make_corpus_trad('/scp-180'))

8

## Traitement du sous-corpus

In [12]:
import spacy
nlp = spacy.load('fr_core_news_md')

c:\Users\Perrine\anaconda3\lib\site-packages\spacy\util.py:887: UserWarning: [W095] Model 'fr_core_news_md' (3.1.0) was trained with spaCy v3.1 and may not be 100% compatible with the current version (3.5.1). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [22]:
seuil=0.50

In [14]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import Normalizer, StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import statistics
import matplotlib.pyplot as plt

In [15]:
def traitement_sous_corpus(corpus,seuil=seuil):
    pipeline = Pipeline([
        ('tfidf', TfidfVectorizer(use_idf=True, norm=None)),
    ])
    matrice = pipeline.fit_transform(corpus)
    cos = cosine_similarity(matrice, matrice)
    np.fill_diagonal(cos, 0)
    cos[cos<seuil]=0
    return(cos[0][1:])

In [16]:
res=traitement_sous_corpus(make_corpus_trad('/scp-180'))

## Faire ces opérations sur l'ensemble des sous-groupes

In [23]:
sum = 0
sum_valid = 0
total_valid_sim = []
for trad_id in metadonnees_en['id']:
    sous_corpus = make_corpus_trad(trad_id)
    if sous_corpus != []:
        sum+=1
        cos_similarity = traitement_sous_corpus(sous_corpus)
        max_sim = max(cos_similarity)
        if max_sim>seuil:
            sum_valid+=1
            total_valid_sim+= [max_sim]
print('Nb traductions : ',sum)
print('Nb traduction qui ont effectivement une similarité élevée :',sum_valid)
print('Moyenne du texte max sur les traductions qui ont effectivement une similarité élevée :',statistics.mean(total_valid_sim))
print('écart-type :',statistics.pstdev(total_valid_sim))

Nb traductions :  2162
Nb traduction qui ont effectivement une similarité élevée : 339
Moyenne du texte max sur les traductions qui ont effectivement une similarité élevée : 0.7593144922512494
écart-type : 0.04304573264652958


## Faire ces opérations sur des sous-corpus aléatoires

In [27]:
import random

In [31]:
sum = 0
sum_valid = 0
total_valid_sim = []
for _ in range(2000):
    sous_corpus = [random.choice(list_textes_en)]
    for i in range(random.choice(range(2,27))):
        sous_corpus += [random.choice(list_textes_fr)]
    if sous_corpus != []:
        sum+=1
        cos_similarity = traitement_sous_corpus(sous_corpus)
        max_sim = max(cos_similarity)
        if max_sim>seuil:
            sum_valid+=1
            total_valid_sim+= [max_sim]
print('Nb traductions : ',sum)
print('Nb traduction qui ont effectivement une similarité élevée :',sum_valid)
print('Moyenne du texte max sur les traductions qui ont effectivement une similarité élevée :',statistics.mean(total_valid_sim))
print('écart-type :',statistics.pstdev(total_valid_sim))

Nb traductions :  2000
Nb traduction qui ont effectivement une similarité élevée : 177
Moyenne du texte max sur les traductions qui ont effectivement une similarité élevée : 0.7465283540835175
écart-type : 0.037421115943572544
